In [1]:
# import the necessary packages
from pyemotionwheel import BasicEmotionWheel
import openai
import json

In [3]:
# set your OpenAI API key
#openai.api_key = "..."

In [8]:
# instantiate an instance of the emotion wheel
wheel = BasicEmotionWheel()

In [4]:
# create a prompt to identify emotions in a journal entry
identify_emotions_prompt_template = """
Act as an expert psychologist who specializes in emotional intelligence.

Your job is to analyze my << JOURNAL ENTRY >> and identify which << EMOTIONS >> are present. These emotions come from my emotion wheel. Any emotions you identify MUST come the << EMOTIONS >> list. If you identify an emotion that is NOT present in the emotion list, use the most similar emotion from the list.

Omit any pre and post-text in your response. Format your response as a JSON list using the << JSON TEMPLATE >>.

<< JOURNAL ENTRY >>
{journal_entry}

<< EMOTIONS >>
{emotion_list}

<< JSON TEMPLATE >>
{
	"emotions": []
}
"""

In [5]:
# define our journal entry, which was generated by ChatGPT using the following prompt: 
#
# Act as an angsty emo teenager who's girlfriend just broke up with him at lunch. Write his journal entry that evening.
#
# (this prompt may or may not be inspired my real-life events...)
journal_entry = """
Tuesday, November 6th

It's like the whole world just flipped into grayscale. She broke up with me. Right there in the cafeteria, with the buzz of a hundred conversations around us. It felt like someone sucked all the air out of the room. I could barely hear her over the noise in my head, the kind of noise that doesn't have a sound but still screams at you.

I don't get it. Yesterday, we were talking about the band's new song, and she was looking at me with those eyes that made me feel like I was actually somebody. Today, she's handing me back my hoodie like it's some kind of contaminated evidence.

I went through the rest of the day like a ghost. I couldn't focus on the teachers, the lessons, or the homework. It was all just a blur. I kept replaying that moment over and over, looking for some sign, something I missed. Did I say something? Did I not say something? It's like trying to find a piece in a puzzle that just doesn't fit anywhere.

The worst part? The pity looks. Everyone knows. They whisper and steal glances like I'm on display in some twisted museum of heartbreak. I wanted to scream, to tell them all to just shut up, but what's the point? They wouldn't get it. They don't know what it's like to feel everything so deeply that it's like drowning, but instead of fighting for air, you just want to sink.

I can't even escape into music tonight. Every lyric seems like it's mocking me, every chord is a reminder of her. I tried writing something, anything. All that came out were scribbles and scratched-out lines that looked more like the ravings of a madman than any kind of poetry or song.

Mom knocked on my door earlier, asked if I'm okay. I told her I'm fine, just tired. I'm not fine. I'm so far from fine it's like I'm on a different planet. And I'm tired, but not the kind of tired sleep can fix. I'm tired in my bones, in my soul.

I don't know what to do with all this... this rage, this sadness, this... emptiness. She was a part of my day, every day, and now there's just this hole where she used to be. It's like someone carved out a piece of me, and I'm just supposed to go on like everything's normal.

But it's not. Nothing's normal. And the worst part is, tomorrow, I have to get up and do it all over again. Walk the same halls, see the same faces, and pretend like I didn't just get ripped in half.

For now, I'll just sit here in the dark, with my guitar lying next to me, unplayed. Maybe sleep will come, maybe it won't. Maybe I'll wake up and this will all just be some bad dream.

But who am I kidding? This is real, and it's not going anywhere. And neither am I.
"""

In [9]:
# build the prompt to identify emotions
emotions_list = [str(e) for e in wheel.all_emotions()]
prompt = identify_emotions_prompt_template.replace(
    "{journal_entry}", journal_entry)
prompt = prompt.replace("{emotion_list}", ",".join(emotions_list))
print(prompt)


Act as an expert psychologist who specializes in emotional intelligence.

Your job is to analyze my << JOURNAL ENTRY >> and identify which << EMOTIONS >> are present. These emotions come from my emotion wheel. Any emotions you identify MUST come the << EMOTIONS >> list. If you identify an emotion that is NOT present in the emotion list, use the most similar emotion from the list.

Omit any pre and post-text in your response. Format your response as a JSON list using the << JSON TEMPLATE >>.

<< JOURNAL ENTRY >>

Tuesday, November 6th

It's like the whole world just flipped into grayscale. She broke up with me. Right there in the cafeteria, with the buzz of a hundred conversations around us. It felt like someone sucked all the air out of the room. I could barely hear her over the noise in my head, the kind of noise that doesn't have a sound but still screams at you.

I don't get it. Yesterday, we were talking about the band's new song, and she was looking at me with those eyes that mad

In [12]:
# submit the emotion identification prompt to OpenAI
completion = openai.chat.completions.create(
    model="gpt-4",
    temperature=0.7,
    messages=[{
        "role": "user",
        "content": prompt
    }]
)
blob = json.loads(completion.choices[0].message.content)
identified_emotions = blob["emotions"]
identified_emotions

['Sadness',
 'Depression',
 'Despair',
 'Rage',
 'Frustrated',
 'Disappointed',
 'Isolated',
 'Lonely',
 'Neglected',
 'Powerless',
 'Suffering',
 'Agony',
 'Hurt',
 'Confused',
 'Disillusioned']

In [13]:
# initialize a dictionary of emotion hierachies
emotion_hierarchies = {}

# loop over all identified emotions
for emotion_name in identified_emotions:
    # find the current emotion
    emotion = wheel.find_emotion(emotion_name)

    # update the emotion hierarchies by including (1) the path back to the
    # root, and (2) any children emotions
    emotion_hierarchies[emotion_name] = {
        "path": [str(e) for e in emotion.path[::-1]],
        "children": [str(e) for e in emotion.children],
    }

# display the hierachies
emotion_hierarchies    

{'Sadness': {'path': ['Sadness', 'root'],
  'children': ['Suffering',
   'Despondent',
   'Disappointed',
   'Shameful',
   'Neglected',
   'Despair']},
 'Depression': {'path': ['Depression', 'Despondent', 'Sadness', 'root'],
  'children': []},
 'Despair': {'path': ['Despair', 'Sadness', 'root'],
  'children': ['Grief', 'Powerless']},
 'Rage': {'path': ['Rage', 'Anger', 'root'], 'children': ['Hate', 'Hostile']},
 'Frustrated': {'path': ['Frustrated', 'Exasperated', 'Anger', 'root'],
  'children': []},
 'Disappointed': {'path': ['Disappointed', 'Sadness', 'root'],
  'children': ['Dismayed', 'Displeased']},
 'Isolated': {'path': ['Isolated', 'Neglected', 'Sadness', 'root'],
  'children': []},
 'Lonely': {'path': ['Lonely', 'Neglected', 'Sadness', 'root'],
  'children': []},
 'Neglected': {'path': ['Neglected', 'Sadness', 'root'],
  'children': ['Isolated', 'Lonely']},
 'Powerless': {'path': ['Powerless', 'Despair', 'Sadness', 'root'],
  'children': []},
 'Suffering': {'path': ['Suffering

In [14]:
# create a prompt to help build emotional intelligence (EQ)
eq_prompt_template = """
Below I have included the emotion wheel hierarchy for each of the << IDENTIFIED EMOTIONS >> in my << JOURNAL ENTRY >>.

Using your expert knowledge as a psychologist and therapist to help me:

1. Use emotional intelligence to trace my emotions back to the root
2. Deepen my emotions and move through the children nodes, ideally finding release in the depening

Be specific in your advice, and show your thinking step-by-step. Use the tone and style of an empathetic, caring therapist.

Format your response as Markdown, omitting any pre or post text.

<< JOURNAL ENTRY >>
{journal_entry}

<< IDENTIFIED EMOTIONS >>
{identified_emotions}
"""

In [15]:
# build the emotional intelligence prompt
prompt = eq_prompt_template.replace("{journal_entry}", journal_entry)
prompt = prompt.replace("{identified_emotions}",
                        json.dumps(emotion_hierarchies))
print(prompt)


Below I have included the emotion wheel hierarchy for each of the << IDENTIFIED EMOTIONS >> in my << JOURNAL ENTRY >>.

Using your expert knowledge as a psychologist and therapist to help me:

1. Use emotional intelligence to trace my emotions back to the root
2. Deepen my emotions and move through the children nodes, ideally finding release in the depening

Be specific in your advice, and show your thinking step-by-step. Use the tone and style of an empathetic, caring therapist.

Format your response as Markdown, omitting any pre or post text.

<< JOURNAL ENTRY >>

Tuesday, November 6th

It's like the whole world just flipped into grayscale. She broke up with me. Right there in the cafeteria, with the buzz of a hundred conversations around us. It felt like someone sucked all the air out of the room. I could barely hear her over the noise in my head, the kind of noise that doesn't have a sound but still screams at you.

I don't get it. Yesterday, we were talking about the band's new s

In [16]:
# submit the emotional intelligence prompt to OpenAI
completion = openai.chat.completions.create(
    model="gpt-4",
    temperature=0.7,
    messages=[{
        "role": "user",
        "content": prompt
    }]
)
response = completion.choices[0].message.content
print(response)

# Unpacking Emotions

This is clearly a hard time for you, and I want to commend you for sharing your feelings and experiences. By identifying your emotions, you've already taken a significant step towards understanding your emotional response to this situation.

## Tracing Back to Root Emotions

Starting with **sadness**, it seems to be a primary emotion here. The breakup has left you feeling sad and downcast. This is not surprising, as you've lost something significant in your life. Sadness is a natural response to loss. The fact that you're also experiencing **depression** and **despair** tells me that this sadness is deep and pervasive. You're feeling the weight of the loss, and it's impacting your daily life.

Moving on to **anger**, this emotion is often a response to feeling wronged or slighted. You mention feeling **rage** and **frustration**, which could be directed towards the situation, your ex-partner, or even yourself. It's important to remember that anger, while uncomfort